In [59]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import itertools
from shapely.geometry import Point, LineString
pd.set_option('display.max_rows', 150)
#plt.style.use('')
import numpy as np
import re 


In [60]:
files = {'viajes':'EDM2018VIAJES','individuos':'EDM2018INDIVIDUOS','hogares':'EDM2018HOGARES','etapas':'EDM2018XETAPAS'}
schemas = {
    'etapas':{
        'ID_HOGAR':int,
        'ID_IND':int,
        'ID_VIAJE':int,
        'ID_ETAPA':int,
        'EMODO':int,
        'EMODO1':str,
        'MODO_PRIORITARIO':int,
        'MODO':int,
        'ELINEAEMPRESA_ORIGINAL':str,
        'ESUBIDA':str,
        'ESUBIDA_cod':str,
        'EBAJADA':str,
        'EBAJADA_cod':str,
        'ETITULO':str,
        'EESTACIONA':str,
        'EOCUPACION':str,
        'ETDESPH':str,
        'TIPO_ENCUESTA':str,
        'COD_MUN_PARADA':str,
        'ELE_G_POND_Esc2':float},
    'viajes':{
        'ID_HOGAR': int,
        'ID_IND': int,
        'ID_VIAJE': int,
        'VORI': int,
        'VORIHORAINI': int,
        'VDES': int,
        'VDESHORAFIN': int,
        'VFRECUENCIA': int,
        'VVEHICULO': float,
        'VNOPRIVADO': float,
        'VNOPUBLICO': float,
        'VORIZT1259': str,
        'VDESZT1259': str,
        'TIPO_ENCUESTA': str,
        'N_ETAPAS_POR_VIAJE': int,
        'MOTIVO_PRIORITARIO': int,
        'DISTANCIA_VIAJE': float,
        'MODO_PRIORITARIO': int,
        'ELE_G_POND_ESC2': float},
    'individuos':{
        'ID_HOGAR':int,
        'ID_IND':int,
        'C2SEXO':int,
        'EDAD_FIN':int,
        'ELE_G_POND':float,
        'C4NAC':int,
        'C5CAM':int,
        'C6CARNE':int,
        'C7ESTUD':int,
        'C8ACTIV':int,
        'C9PROF':str,
        'C10SECTOR':str,
        'C13TARJETA':int,
        'C14ABONO':str,
        'DDIA':int,
        'DMES':int,
        'DANNO':int,
        'DIASEM':int,
        'DNOVIAJO':str,
        'C11ZT1259':str,
        'C12ZT1259':str,
        'CPMR':int,
        'TIPO_ENCUESTA':str},
    'hogares':{
        'ID_HOGAR':int,
        'CODMUNI':int,
        'NOMMUNI':str,
        'CODPROV':str,
        'NOMPROV':str,
        'ZT1259':str,
        'CZ208':str,
        'ELE_HOGAR_NUEVO':float,
        'TIPO_ENCUESTA':str,
        'A1PER':int,
        'A2PER4':int,
        'B1NVE':int,
        'V1B11TIPO':str,
        'V1B12CARB':str,
        'V1B13EST':str,
        'V2B11TIPO1':str,
        'V2B12CARB1':str,
        'V2B13EST1':str,
        'V3B11TIPO1':str,
        'V3B12CARB1':str,
        'V3B13EST1':str,
        'V4B11TIPO1':str,
        'V4B12CARB1':str,
        'V4B13EST1':str,
        'V5B11TIPO1':str,
        'V5B12CARB1':str,
        'V5B13EST1':str,
        'N_MIEMBROS_POR_HOGAR':int,
        'N_VIAJES_POR_HOGAR':int},
        }

In [61]:
path = r"C:\Users\ManuBenito\Documents\Walknet-DataLake\sources\edm2018\level0"

In [62]:
data = {n:pd.read_excel(
    r"{path}\trips\{p}.xlsx".format(path=path,p=p), sheet_name = n.upper(),dtype=schemas[n]
    ) for n,p in files.items()}


In [77]:
codes = {n:pd.read_excel(
    r"{path}\trips\{p}.xlsx".format(path=path,p=p), sheet_name = 'LIBRO DE CODIGOS', header = 0, usecols = ['VARIABLE','VALORES'],skiprows=[1,2], engine ='openpyxl'
    ) for n,p in files.items()}

In [78]:
def fill_in_order(df):
    fill = ''
    for i,r in df.iterrows():
        if not pd.isnull(r['VARIABLE']):
            fill = r['VARIABLE']
        else:
            df.at[i,'VARIABLE'] = fill
    return df


In [79]:
codes = {k:fill_in_order(df.query('VALORES.notnull()', engine='python')) for k,df in codes.items()}

In [80]:
def parse_codes(df):
    df['CODIGO'] = df['VALORES'].str.extract(r"""(?m)^(\d+).*""").astype(int)
    df['LITERAL'] = df['VALORES'].str.split(r"""^[^A-Z]*""").str[1].str.strip().str.removesuffix(r"""'""").str.strip()
    return df[['VARIABLE','CODIGO','LITERAL']]

In [81]:
codes = {k:parse_codes(df) for k,df in codes.items()}

In [82]:
codes = {n: {k : g.LITERAL.to_dict() for k, g in df.set_index('CODIGO').groupby('VARIABLE')} for n,df in codes.items()}

In [84]:
def map_columns(df,mapper):
    for c in [x for x in df.columns if x in mapper.keys()]:
        df[c] = df[c].replace(mapper[c])
    return df

In [85]:
data = {k:map_columns(df,codes[k]) for k,df in data.items()}

In [87]:
final = pd.merge(data['etapas'],data['viajes'], on=['ID_HOGAR','ID_IND','ID_VIAJE'], how='left', suffixes=("","_x"))
final = final[[c for c in final.columns if not c.endswith("_x")]]

In [88]:
final = pd.merge(data['individuos'], final, on=['ID_HOGAR','ID_IND'], how='right',suffixes=("","_x"))
final = final[[c for c in final.columns if not c.endswith("_x")]]

In [89]:
final = pd.merge(data['hogares'],final, on=['ID_HOGAR'], how='left',suffixes=("","_x"))
final = final[[c for c in final.columns if not c.endswith("_x")]]

In [ ]:
for c in final.columns:
    if final[c].dtype == str:
        final[c] = final[c].fillna("")
    else:
        final[c] = final[c].fillna(0)    

In [ ]:
#data.set_index(['ID_HOGAR','ID_IND','ID_VIAJE'],inplace=True)

In [ ]:
#data.to_csv(r"C:\Users\ManuBenito\Documents\Walknet-DataLake\sources\spain\mobility\edm2018\level1\level1_edm2018_con_etapas.csv",sep=";")

In [ ]:
data = pd.read_csv(r"C:\Users\ManuBenito\Documents\Walknet-DataLake\sources\spain\mobility\edm2018\level1\level1_edm2018.csv",sep=";")

In [ ]:
data.info()

In [ ]:
len(data)

In [ ]:
data.MODO_PRIORITARIO.unique()

In [ ]:
ped = data.loc[data['MODO_PRIORITARIO'] =='Andando/pie']

In [ ]:
ped['ONE'] = 1

In [ ]:
sns.countplot(y="MOTIVO_PRIORITARIO", data=ped)

In [ ]:
sns.countplot(y="VNOPUBLICO", data=ped)

In [ ]:
sns.countplot(y="VNOPRIVADO", data=ped,orient='v')

In [ ]:
sns.countplot(y="C7ESTUD", data=ped,orient='v')

In [ ]:
sns.countplot(y="C8ACTIV", data=ped,orient='v')

In [ ]:
sns.countplot(y="VFRECUENCIA", data=ped,orient='v')

In [ ]:
sns.countplot(x="C2SEXO", data=ped)

In [ ]:
figure = plt.figure(figsize=(5.2,3.7))
plt.hist([ped[ped['C2SEXO']=='Mujer']['VORIHORAINI'], ped[ped['C2SEXO']=='Hombre']['VORIHORAINI']], stacked=False, bins=24, label=['Mujer','Hombre'])
plt.xlabel('Hour')
plt.ylabel('Sex')
plt.legend()

In [ ]:
ped.MOTIVO_PRIORITARIO.unique()

In [ ]:
figure = plt.figure(figsize=(5.2,3.7))
plt.hist([ped[ped['MOTIVO_PRIORITARIO']=='Compras']['EDAD_FIN'],
          ped[ped['MOTIVO_PRIORITARIO']=='Ocio']['EDAD_FIN'],
          ped[ped['MOTIVO_PRIORITARIO']=='Estudio']['EDAD_FIN'],
          ped[ped['MOTIVO_PRIORITARIO']=='Deporte/ dar un paseo']['EDAD_FIN'],
          ped[ped['MOTIVO_PRIORITARIO'].isin(['Acompañamiento a otra persona','Médico'])]['EDAD_FIN'],
          ped[ped['MOTIVO_PRIORITARIO'].isin(['Casa','Trabajo','Gestión de trabajo'])]['EDAD_FIN'],
         ped[ped['MOTIVO_PRIORITARIO'].isin(['Asunto personal','Otro domicilio','Otros'])]['EDAD_FIN']],
         stacked=True, bins=100, label=['Compras','Ocio','Estudio','Deporte/Paseo','Cuidados','Trabajo','Otros'])
plt.xlabel('Age')
plt.ylabel('Motivo')
plt.legend()
#plt.savefig("C:\\Users\\manub\\DATA\\00-EXTERNO\MANU\TODOS_MOTIVO_EDAD.png")

In [ ]:
ped_muj = ped.loc[ped['C2SEXO']=='Mujer']
ped_hom = ped.loc[ped['C2SEXO']=='Hombre']

In [ ]:
figure = plt.figure(figsize=(5.2,3.7))
axes = plt.gca()
axes.set_ylim([0,2500])
plt.hist([ped_muj[ped_muj['MOTIVO_PRIORITARIO']=='Compras']['EDAD_FIN'],
          ped_muj[ped_muj['MOTIVO_PRIORITARIO']=='Ocio']['EDAD_FIN'],
          ped_muj[ped_muj['MOTIVO_PRIORITARIO']=='Estudio']['EDAD_FIN'],
          ped_muj[ped_muj['MOTIVO_PRIORITARIO']=='Deporte/ dar un paseo']['EDAD_FIN'],
          ped_muj[ped_muj['MOTIVO_PRIORITARIO'].isin(['Acompañamiento a otra persona','Médico'])]['EDAD_FIN'],
          ped_muj[ped_muj['MOTIVO_PRIORITARIO'].isin(['Casa','Trabajo','Gestión de trabajo'])]['EDAD_FIN'],
         ped_muj[ped_muj['MOTIVO_PRIORITARIO'].isin(['Asunto personal','Otro domicilio','Otros'])]['EDAD_FIN']],
         stacked=True, bins=100, label=['Compras','Ocio','Estudio','Deporte/Paseo','Cuidados','Trabajo','Otros'])
plt.xlabel('Age')
plt.ylabel('Motivo')
plt.legend()
#plt.savefig("C:\\Users\\manub\\DATA\\00-EXTERNO\MANU\MUJERES_MOTIVO_EDAD.png")

In [ ]:
figure = plt.figure(figsize=(5.2,3.7))
axes = plt.gca()
axes.set_ylim([0,2500])
plt.hist([ped_hom[ped_hom['MOTIVO_PRIORITARIO']=='Compras']['EDAD_FIN'],
          ped_hom[ped_hom['MOTIVO_PRIORITARIO']=='Ocio']['EDAD_FIN'],
          ped_hom[ped_hom['MOTIVO_PRIORITARIO']=='Estudio']['EDAD_FIN'],
          ped_hom[ped_hom['MOTIVO_PRIORITARIO']=='Deporte/ dar un paseo']['EDAD_FIN'],
          ped_hom[ped_hom['MOTIVO_PRIORITARIO'].isin(['Acompañamiento a otra persona','Médico'])]['EDAD_FIN'],
          ped_hom[ped_hom['MOTIVO_PRIORITARIO'].isin(['Casa','Trabajo','Gestión de trabajo'])]['EDAD_FIN'],
         ped_hom[ped_hom['MOTIVO_PRIORITARIO'].isin(['Asunto personal','Otro domicilio','Otros'])]['EDAD_FIN']],
         stacked=True, bins=100, label=['Compras','Ocio','Estudio','Deporte/Paseo','Cuidados','Trabajo','Otros'])
plt.xlabel('Age')
plt.ylabel('Motivo')
plt.legend()
#plt.savefig("C:\\Users\\manub\\DATA\\00-EXTERNO\MANU\HOMBRES_MOTIVO_EDAD.png")

In [ ]:
sns.countplot(y="C8ACTIV", data=ped,orient='v')

In [ ]:
ped['EDAD_FIN'].hist(bins=100)

In [ ]:
sns.displot(ped, x="EDAD_FIN", hue="MOTIVO_PRIORITARIO", kind="kde", multiple="stack",)

In [ ]:
sns.displot(ped.loc[ped["DISTANCIA_VIAJE"]<5], x="DISTANCIA_VIAJE", hue="MOTIVO_PRIORITARIO", kind="kde", multiple="stack")

In [ ]:
sns.displot(ped.loc[ped["DISTANCIA_VIAJE"]<2], x="DISTANCIA_VIAJE", hue="VNOPRIVADO", kind="kde", multiple="stack")

In [ ]:
sns.displot(ped.loc[ped["DISTANCIA_VIAJE"]<2], x="DISTANCIA_VIAJE", hue="VNOPUBLICO", kind="kde", multiple="stack")

In [ ]:
sns.displot(data.loc[data["DISTANCIA_VIAJE"]<5], x="DISTANCIA_VIAJE", hue="MODO_PRIORITARIO", kind="kde", multiple="stack")

In [ ]:
sns.displot(data.loc[data["DISTANCIA_VIAJE"]<5], x="DISTANCIA_VIAJE", hue="MOTIVO_PRIORITARIO", kind="kde", multiple="stack")

## Zonas

In [ ]:
zonas = gpd.read_file(r"C:\Users\ManuBenito\Documents\Walknet-DataLake\sources\spain\mobility\edm2018\level0\spatialdata\ZonificacionZT1259.geojson")
zonas = zonas.to_crs({'init': 'epsg:25830'})
#zonas_point = zonas.copy()
#zonas_point['geometry'] = zonas_point['geometry'].representative_point()

In [ ]:
zonas.columns

In [ ]:
zonas.plot(column='ZT1259')

In [ ]:
usos = gpd.read_file(r"C:\Users\ManuBenito\Documents\Walknet-DataLake\sources\spain\landuse\catastro\level1\PORTALES_AMM_CDATA.shp")

In [ ]:
parcelas = gpd.read_file(r"C:\Users\ManuBenito\Documents\Walknet-DataLake\sources\spain\landuse\catastro\level1\PARCELAS_AMM_USOS.shp")

In [ ]:
clc = gpd.read_file(r"C:\Users\ManuBenito\Documents\Walknet-DataLake\sources\spain\landuse\clc\CLC18_AMM.shp")

In [ ]:
clc = clc.loc[clc['Code_18'].isin(['111','112','121'])]

In [ ]:
clc = clc.dissolve(by='Cod_Prov')

In [ ]:
usos=usos.to_crs({'init': 'epsg:25830'})
parcelas=parcelas.to_crs({'init': 'epsg:25830'})
clc=clc.to_crs({'init': 'epsg:25830'})

In [ ]:
#res_intersection = gpd.overlay(parcelas, clc, how='intersection')
#res_intersection = res_intersection.dissolve(by='REFCAT',as_index=False)
#res_intersection.to_file(r"C:\Users\ManuBenito\Documents\Walknet-DataLake\sources\spain\landuse\catastro\level1\developed.gpkg",driver='GPKG')

In [ ]:
parcelas_developed = gpd.read_file(r"C:\Users\ManuBenito\Documents\Walknet-DataLake\sources\spain\landuse\catastro\level1\developed.gpkg")

In [ ]:
#parcelas_developed = res_intersection.copy()

In [ ]:
parcelas_developed

In [ ]:
parcelas_developed['area_ha']=(parcelas_developed['geometry'].area)/10000

In [ ]:
parcelas_developed = parcelas_developed[['REFCAT','area_ha','geometry']]

In [ ]:
#zonas_amm = zonas.loc[zonas['AMM']==1]
zonas_amm = zonas.copy()

In [ ]:
#usos.loc[usos['PCAT'].isin(parcelas_developed['REFCAT'].unique())]

In [ ]:
poblacion = pd.read_parquet(r"C:\Users\ManuBenito\Documents\Walknet-DataLake\sources\spain\demographics\ine_adrh\ine_atlasrenta\level1_spain_ine_atlasrenta_census tract_2018.parquet")
geometry = gpd.read_file(r"C:\Users\ManuBenito\Documents\Walknet-DataLake\sources\spain\demographics\level1_geo_census tract_code_2018.gpkg")

In [ ]:
poblacion = pd.merge(geometry,poblacion,on='census tract_code_2018',how='left',suffixes=('',"_x"))
poblacion = poblacion[poblacion['province_code_2018']=='28']


In [ ]:
poblacion = poblacion[['Indicadores demográficos_units_Edad media de la población_Personas,   %,   Años_2018',
'Indicadores demográficos_units_Población_Personas,   %,   Años_2018',
'Indicadores demográficos_units_Porcentaje de hogares unipersonales_Personas,   %,   Años_2018',
'Indicadores demográficos_units_Porcentaje de población de 65 y más años_Personas,   %,   Años_2018',
'Indicadores demográficos_units_Porcentaje de población menor de 18 años_Personas,   %,   Años_2018',
'Indicadores demográficos_units_Tamaño medio del hogar_Personas,   %,   Años_2018',
'Indicadores de renta media y mediana_units_Renta bruta media por hogar_€_2018',
'Indicadores de renta media y mediana_units_Renta bruta media por persona_€_2018',
'Indicadores de renta media y mediana_units_Renta neta media por hogar_€_2018',
'Indicadores de renta media y mediana_units_Renta neta media por persona_€_2018',
'Indicadores de renta media y mediana_units_Media de la renta por unidad de consumo_€_2018',
'Indicadores de renta media y mediana_units_Mediana de la renta por unidad de consumo_€_2018',
'Indicadores de renta media y mediana_units_Renta mediana por hogar_€_2018','geometry']]

In [ ]:
poblacion = poblacion.to_crs({'init': 'epsg:25830'})

In [ ]:
poblacion

In [ ]:
poblacion.plot(column='Indicadores de renta media y mediana_units_Media de la renta por unidad de consumo_€_2018')

In [ ]:
usos_poblacion = gpd.sjoin(usos,poblacion,how='left',op='intersects')

In [ ]:
fig,ax = plt.subplots(figsize=(20,20))
usos_poblacion.plot(markersize=0.05,ax=ax,c='black',zorder=2)
zonas_amm.plot(ax=ax,zorder=0)
parcelas_developed.plot(ax=ax,zorder=1,color='red')

In [ ]:
usos_poblacion = usos_poblacion.replace(-999,np.nan)

In [ ]:
usos_poblacion['POBLACION_TOTAL'] = usos_poblacion['NV'] * usos_poblacion['Indicadores demográficos_units_Tamaño medio del hogar_Personas,   %,   Años_2018']
usos_poblacion['TAMANO HOGAR'] = usos_poblacion['Indicadores demográficos_units_Tamaño medio del hogar_Personas,   %,   Años_2018']
usos_poblacion['RENTA_TOTAL'] = usos_poblacion['Indicadores de renta media y mediana_units_Renta bruta media por persona_€_2018'] * usos_poblacion['POBLACION_TOTAL']
usos_poblacion['MENORES_TOTAL'] = (usos_poblacion['POBLACION_TOTAL']*usos_poblacion['Indicadores demográficos_units_Porcentaje de población menor de 18 años_Personas,   %,   Años_2018'])/100
usos_poblacion['MAYORES_TOTAL'] = (usos_poblacion['POBLACION_TOTAL']*usos_poblacion['Indicadores demográficos_units_Porcentaje de población de 65 y más años_Personas,   %,   Años_2018'])/100

In [ ]:
len(usos_poblacion)

In [ ]:
#usos_poblacion.drop(columns=['index_right'],inplace=True)

In [ ]:
zonas_usos_poblacion= gpd.sjoin(zonas_amm,usos_poblacion,op='intersects',how='right')

In [ ]:
map_usos = {'Viviendas':['V'],'Compras':['C'],'Ocio':['G'],'Estudio':['E'],'Deporte/Paseo':['K'],'Cuidados':['Y','E'],'Trabajo':['C','O','P']}

In [ ]:
zonas_usos_poblacion = zonas_usos_poblacion.groupby('ZT1259').sum()[['NA', 'NB', 'NC',
       'NE', 'NG', 'NI', 'NJ', 'NK', 'NM', 'NO', 'NP', 'NR', 'NT', 'NV', 'NY',
       'NZ', 'SA', 'SB', 'SC', 'SE', 'SG', 'SI', 'SJ', 'SK', 'SM', 'SO', 'SP',
       'SR', 'ST', 'SV', 'SY', 'SZ', 'SCONS','POBLACION_TOTAL','RENTA_TOTAL',
       'MENORES_TOTAL', 'MAYORES_TOTAL']]


In [ ]:
zonas_usos_poblacion['RENTA_MEDIA'] = zonas_usos_poblacion['RENTA_TOTAL'] / zonas_usos_poblacion['POBLACION_TOTAL']
zonas_usos_poblacion['PROP_MAYORES'] = (zonas_usos_poblacion['MAYORES_TOTAL']*100) / zonas_usos_poblacion['POBLACION_TOTAL']
zonas_usos_poblacion['PROP_MENORES'] = (zonas_usos_poblacion['MENORES_TOTAL']*100) / zonas_usos_poblacion['POBLACION_TOTAL']

In [ ]:
clc.plot()

In [ ]:
clc = clc[['geometry']]

In [ ]:
developed = gpd.overlay(zonas_amm, clc, how='intersection')

In [ ]:
developed['DEVELOPED_AREA'] = developed['geometry'].area/10000

In [ ]:
developed_area = developed.groupby('ZT1259').sum()[['DEVELOPED_AREA']]

In [ ]:
developed_area.reset_index(inplace=True)

In [ ]:
zonas_usos_poblacion

In [ ]:
zonas_usos_poblacion.fillna(0,inplace=True)

In [ ]:
'''
wm = lambda x: np.average(x, weights=poblacion.loc[x.index, "Población_2017"])

wmeans1 = poblacion.groupby(["ZT1259"]).agg(Edad_Media_2017=("Edad media de la población_2017", wm))
wmeans2 = poblacion.groupby(["ZT1259"]).agg(Jovenes_2017=("Porcentaje de población menor de 18 años_2017", wm))
wmeans3 = poblacion.groupby(["ZT1259"]).agg(Mayores_2017=("Porcentaje de población de 65 y más años_2017", wm))
'''                                  
#FALTA HACER LA WEIGHTED MEAN DE EDAD

In [ ]:
developed_area

In [ ]:
zonas_usos_poblacion

In [ ]:
zonas_data = pd.merge(zonas_amm,zonas_usos_poblacion,on='ZT1259',how='left')

In [ ]:
zonas_data = pd.merge(zonas_data,developed_area,on='ZT1259',how='left')

In [ ]:
zonas_data

In [ ]:
zonas_data['POBLACION_TOTAL'].fillna(0,inplace=True)
zonas_data['DEVELOPED_AREA'].fillna(0,inplace=True)

In [ ]:
zonas_data.plot(column='DEVELOPED_AREA')

In [ ]:
zonas_data['DENSIDAD'] =zonas_data['POBLACION_TOTAL'] / zonas_data['DEVELOPED_AREA']

In [ ]:
zonas_data['DENSIDAD'].fillna(0,inplace=True)

In [ ]:
def simpson_diversity(row,fields):
    all_species = sum([row[a] for a in fields])
    diversity = 1 - sum([(row[a]/all_species)**2 for a in fields if row[a]!=0])
    row['DIVERSIDAD'] = diversity
    return row

In [ ]:
uses = ['SC','SE','SG','SK','SP','SR','ST','SV','SY']

In [ ]:
for a in uses:
    zonas_data[a].fillna(0,inplace=True)

In [ ]:
zonas_data = zonas_data.apply(lambda row:simpson_diversity(row,uses),axis=1)

In [ ]:
zonas_data.loc[zonas_data['DIVERSIDAD']==1,'DIVERSIDAD'] = 0

In [ ]:
zonas_data.to_file(r"C:\Users\ManuBenito\Documents\Walknet-DataLake\sources\zonas_data.gpkg",driver='GPKG')

In [ ]:
zonas_data.columns

In [ ]:
zonas_data.columns

In [ ]:
zonas_data = zonas_data[['ZT1259',
       'NA', 'NB', 'NC', 'NE', 'NG', 'NI', 'NJ', 'NK', 'NM', 'NO',
       'NP', 'NR', 'NT', 'NV', 'NY', 'NZ', 'SA', 'SB', 'SC', 'SE', 'SG', 'SI',
       'SJ', 'SK', 'SM', 'SO', 'SP', 'SR', 'ST', 'SV', 'SY', 'SZ', 'SCONS',
       'POBLACION_TOTAL', 'RENTA_TOTAL', 'MENORES_TOTAL', 'MAYORES_TOTAL',
       'RENTA_MEDIA', 'PROP_MAYORES', 'PROP_MENORES', 'DEVELOPED_AREA',
       'DENSIDAD', 'DIVERSIDAD']]

In [ ]:
data_zonas = pd.merge(data,zonas_data.rename(columns={c:f'{c}_origin' for c in zonas_data.columns if c !='ZT1259'}),left_on='VORIZT1259',right_on='ZT1259',how='left')
data_zonas = pd.merge(data_zonas,zonas_data.rename(columns={c:f'{c}_destination' for c in zonas_data.columns if c !='ZT1259'}),left_on='VDESZT1259',right_on='ZT1259',how='left')

In [ ]:
for c in data_zonas.columns:
    print(c)

In [ ]:
sns.displot(data_zonas.loc[data_zonas["DISTANCIA_VIAJE"]<5], x="DIVERSIDAD", hue="MODO_PRIORITARIO", kind="kde", multiple="stack")

In [ ]:
sns.displot(data_zonas.loc[(data_zonas["DISTANCIA_VIAJE"]<5)&(data_zonas["DENSIDAD"]<1000)], x="DENSIDAD", hue="MODO_PRIORITARIO", kind="kde", multiple="stack")

In [ ]:
sns.displot(data_zonas.loc[data_zonas["DISTANCIA_VIAJE"]<5], x="DIVERSIDAD", hue="MOTIVO_PRIORITARIO", kind="kde", multiple="stack")

In [ ]:
sns.displot(data_zonas.loc[(data_zonas["DISTANCIA_VIAJE"]<5)&(data_zonas["DENSIDAD"]<1000)], x="DENSIDAD", hue="MOTIVO_PRIORITARIO", kind="kde", multiple="stack")

In [ ]:
ped = data_zonas.loc[data_zonas['MODO_PRIORITARIO'] =='Andando/pie']

In [ ]:
sns.displot(ped.loc[(ped["DISTANCIA_VIAJE"]<5)&(ped["DENSIDAD"]<1000)], x="DENSIDAD", hue="MOTIVO_PRIORITARIO", kind="kde", multiple="stack")

In [ ]:
sns.displot(ped.loc[ped["DISTANCIA_VIAJE"]<5], x="DIVERSIDAD", hue="MOTIVO_PRIORITARIO", kind="kde", multiple="stack")

In [ ]:
ped.columns

In [ ]:
sns.displot(ped.loc[(ped["DISTANCIA_VIAJE"]<5)&(ped["DENSIDAD"]<1000)], x="DENSIDAD", hue="MOTIVO_PRIORITARIO", kind="kde", multiple="stack")

In [ ]:
sns.displot(data_zonas, x="RENTA_MEDIA", hue="MODO_PRIORITARIO", kind="kde", multiple="stack")

In [ ]:
sns.displot(data_zonas, x="PROP_MENORES", hue="MODO_PRIORITARIO", kind="kde", multiple="stack")

In [ ]:
d = {'Coche conductor particular': 'Coche',
 'Andando/pie': 'Andando',
 'Autobus interurbano': 'Autobus',
 'Autobus discrecional': 'Autobus',
 'Coche pasajero particular': 'Coche',
 0: 'S/D',
 'Otros': 'S/D',
 'Metro': 'Metro/Ferrocarril',
 'Autobus urbano otro municipio': 'Autobus',
 'Renfe Cercanías': 'Metro/Ferrocarril',
 'Coche conductor empresa': 'Coche',
 'Moto/ciclomotor particular': 'Moto',
 'Bicicleta particular': 'Bicicleta',
 'Coche pasajero empresa': 'Coche',
 'Autobus urbano Madrid EMT': 'Autobus',
 'Resto renfe': 'Metro/Ferrocarril',
 'Taxi': 'Coche',
 'Metro ligero/tranvía': 'Metro/Ferrocarril',
 'Coche pasajero alquiler con conductor': 'Coche',
 'Autobus de largo recorrido': 'Autobus',
 'Coche conductor alquiler sin conductor': 'Coche',
 'Moto/ciclomotor empresa': 'Moto',
 'Bicicleta publica': 'Bicicleta',
 'Bicicleta empresa': 'Bicicleta',
 'Moto/ciclomotor publica': 'Moto'}

In [ ]:
data_zonas['MODO_SIMPLIFICADO'] = data_zonas['MODO_PRIORITARIO'].replace(d)

In [ ]:
m ={'Trabajo': 'Trabajo',
 'Compras': 'Compras',
 'Ocio': 'Deporte/ Paseo / Ocio',
 'Asunto personal': 'Otros',
 'Estudio': 'Estudio',
 'Otros': 'Otros',
 0: 'Otros',
 'Gestión de trabajo': 'Trabajo',
 'Deporte/ dar un paseo': 'Deporte/ Paseo / Ocio',
 'Médico': 'Cuidados',
 'Casa': 'Otros',
 'Otro domicilio': 'Otros',
 'Acompañamiento a otra persona': 'Cuidados'}

In [ ]:
data_zonas['MOTIVO_SIMPLIFICADO'] = data_zonas['MOTIVO_PRIORITARIO'].replace(m)

In [ ]:
sns.displot(data_zonas.loc[data_zonas['DISTANCIA_VIAJE']<5], x="DIVERSIDAD", hue="MOTIVO_SIMPLIFICADO", kind="kde", multiple="stack")

In [ ]:
df = pd.DataFrame({'value': np.random.randint(1, 80, 20)}) 
df['group'] = pd.cut(df.value,
                     bins=[0, 5, 31, 51, 80],
                     labels=["very short", "short", "long", "very long"]

In [ ]:
l = list(itertools.combinations(zonas.keys(),2))

In [ ]:
l = ["/".join(list(t)) for t in l]
p = ["/".join([m,m])for m in zonas.keys()]

In [ ]:
p

In [ ]:
lineas = {}
puntos = {}
for el in l:
    l1 = el.split("/")[0]
    l2 = el.split("/")[1]
    line = LineString([zonas[l1],zonas[l2]])
    lineas.update({el:line})
for el in p:
    p0 = el.split("/")[0]
    point = zonas[p0]
    puntos.update({el:point})

In [ ]:
z = pd.DataFrame.from_dict(lineas,orient='index')
x = pd.DataFrame.from_dict(puntos,orient='index')

In [ ]:
d['MODO_PRIORITARIO'].unique()

In [ ]:
#activos = d.loc[d['MODO_PRIORITARIO'].isin(["'24. Andando/pie '"])]
#activos = d.loc[d['MODO_PRIORITARIO'].isin(["'24. Andando/pie '","'21. Bicicleta publica'","'22. Bicicleta empresa'","'20. Bicicleta particular'"])]
#activos = d.loc[d['MODO_PRIORITARIO'].isin(["'11. Coche conductor particular'","'14. Coche pasajero particular'","'12. Coche conductor empresa'","'10. Taxi '","'17. Moto/ciclomotor particular'","'10. Taxi '","'16. Coche pasajero alquiler con conductor'","'13. Coche conductor alquiler sin conductor'","'19. Moto/ciclomotor empresa'","'18. Moto/ciclomotor publica'"])]
activos = d.loc[d['MODO_PRIORITARIO'].isin(["'2. Autobus interurbano '", "'8. Autobus discrecional '","'3. Autobus urbano otro municipio '","'6. Autobus urbano Madrid EMT '","'9. Autobus de largo recorrido '"])]
#activos = d.loc[d['MODO_PRIORITARIO'].isin(["'4. Metro '","'1. Renfe Cercanías '","'7. Resto renfe '","'5. Metro ligero/tranvía '"])]

In [ ]:
co = activos.groupby(['LINEA']).count()
co['COUNT'] = co['ID_HOGAR']
co = co[['COUNT']]
dis = activos.groupby(['LINEA']).mean()
dis = dis[['DISTANCIA_VIAJE']]

In [ ]:
#co.reset_index(inplace=True)
#dis.reset_index(inplace=True)

In [ ]:
z['LINEA']=z.index
z['geometry'] = z[0]
#z.reset_index(inplace=True)
z=z[['LINEA','geometry']]
x['LINEA']=x.index
x['geometry'] = x[0]
#x.reset_index(inplace=True)
x=x[['LINEA','geometry']]

In [ ]:
#lfin = pd.concat([z,co,dis],axis=1)
#pfin = pd.concat([x,co,dis],axis=1)

In [ ]:
lfin = pd.merge(z,co,left_index=True,right_index=True)
lfin = pd.merge(lfin,dis,left_index=True,right_index=True)

In [ ]:
pfin = pd.merge(x,co,left_index=True,right_index=True)
pfin = pd.merge(pfin,dis,left_index=True,right_index=True)

In [ ]:
geo_lfin = gpd.GeoDataFrame(lfin,geometry='geometry',crs='epsg:3857')
geo_pfin = gpd.GeoDataFrame(pfin,geometry='geometry',crs='epsg:3857')

In [ ]:
zonas = gpd.read_file("C:\\Users\\manub\\DATA\\00-EXTERNO\\MANU\\ZonificacionZT1259-shp\\ZonificacionZT1259.shp")
#Visualizamos el dataset
fig, ax = plt.subplots(figsize =(10,10))
zonas.plot(ax=ax,color='black',linewidth=1)
geo_lfin.plot(ax=ax,color='aqua',linewidth=geo_lfin['COUNT']/20,scheme='quantiles',zorder=1)
geo_pfin.plot(ax=ax,color='white',markersize=geo_pfin['COUNT']/5,zorder=2)
#plt.grid(True,color='pink', linestyle='-', linewidth=0.7)
ax.axis("off")
plt.tight_layout()
plt.savefig("C:\\Users\\manub\\DATA\\00-EXTERNO\\MANU\\bus.jpeg",dpi=300)
plt.show();
plt.clf()

#gold,lawngreen,aqua,deeppink